In [1]:
!pip install sent2vec

     |████████████████████████████████| 2.0MB 5.9MB/s 
     |████████████████████████████████| 3.2MB 28.0MB/s 
     |████████████████████████████████| 890kB 40.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=814f5282f451fae71455c949e8ac95019b1bdd534b1d2f0849252566c4af883b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import os.path
from os import path

import spacy
import string
import math
from scipy import spatial
from spacy import displacy
import gensim
from google.colab import files
#################################################
#               STEP 1 START                    #
#################################################
# while(not path.exists("Trec_microblog11.txt")):
  # print("Please upload the file: Trec_microblog11.txt")
  # uploaded = files.upload()

# The core spacy object that will be used for tokenization
sp = spacy.load('en')

# If the above causes an error after installing the package described in (2), install the package described
# in the Note section within the introduction and run this line of code instead of the above.
#sp = spacy.load('en_core_web_sm')

sp.max_length = 6000000

file_1 = open("Trec_microblog11.txt", mode='r', encoding='utf-8-sig')

collection = file_1.readlines()

In [3]:
def preprocess(collection):
  preprocessed = []
  for line in collection:
    tempText = []
    num, _, rest = line.partition("\t")
    tokens = sp(rest, disable=["tagger", "parser", "ner"] )
    rest = rest.lower()
    tempText = [num]
    for token in tokens:
      if not (token.is_stop or token.is_punct or token.is_space or len(token)<=1 or "http://" in token.text or "/"in token.text or "_" in token.text or "." in token.text or "-" in token.text ):
        temp = (token.text.strip("@").strip('#').strip('"').strip("=").replace('"',"")).lower()
        tempText.append(temp)
    preprocessed.append(tempText)
  return preprocessed

preprocessedLines = preprocess(collection)

In [4]:
from sent2vec.vectorizer import Vectorizer
from scipy import spatial
# forming a sentence from a list of tokens
sentences = [' '.join(line[1:]) for line in preprocessedLines ]
# storing corresponding document IDs
# indices = [line[0] for line in preprocessedLines ]

In [5]:
file_2 = open("topics_MB1-49.txt", mode='r', encoding='utf-8-sig')
collection_2 = file_2.readlines()

queries = []

#add queries to list in order
for line in collection_2:
  if "<title>" in line:
    temp = line.split()
    temp = ' '.join(temp[1:len(temp)-1])
    queries.append(temp.strip("@").strip('#').strip("=").replace("'","").replace('"',"").replace("-"," ").replace(",",""))

In [7]:
# reading the content from the results file of a previous assignment as we want to rank those documents
file = open('assign1_Results.txt', 'r')
results = file.readlines()

In [9]:
# function to process the query
def process_query(query):

  tokens = sp(query, disable=["tagger", "parser", "ner"] )
  # print(tokens)
  # query = query.lower()
  tempText = []
  for token in tokens:
    if not (token.is_stop or token.is_punct or token.is_space or len(token)<=1 or "http://" in token.text or "/"in token.text or "_" in token.text or "." in token.text or "-" in token.text ):
        temp = (token.text.strip("@").strip('#').strip('"').strip("=").replace('"',"")).lower()
        tempText.append(temp)
  return tempText

In [10]:
from sent2vec.vectorizer import Vectorizer
from scipy import spatial
# results will be stored in this file
writefile =  open('exp1.txt', 'w')

# iterating over all queries
for i in range(len(queries)):
  # print(i)
  query, indices = queries[i], []

  query = ' '.join( process_query(query) )
# for every query, it shows the index from where ranked documents are starting and where they are ending
  start, end = i*1000, (i+1)*1000
# storing the document ids in indices list 
  for j in range(start, end):
    indices.append( results[j].split(' ')[2] )

  sentences = []
# retrieving the documents from processed lines on the basis of ids as
# first they will iterate over indices
  for index in indices:
# iterating over preprocessedLines to search for the relevant index 
    for line in preprocessedLines:
  # if index is matched then sentence is retrieved
      if index == line[0]:
        sentences.append(' '.join(line[1:]))
        break
# adding a test query into a sentence list
  sentences.insert(0,query)
  # initializing a vectorizer
  vectorizer = Vectorizer()
  # training a bert model
  vectorizer.bert(sentences)
  # vectors which represents the each sentence
  vectors = vectorizer.vectors
  # vectorizer.bert(query)
  sim_scores = []
  # similarity score is measured between vector[0](test vector) and all other vectors
  for k in range(1,len(indices)+1):
    sim_scores.append([indices[k-1],1- spatial.distance.cosine(vectors[0], vectors[k])  ])

  rank = 0
  # sorting the sim_scores, with the most similar at start
  sim_scores.sort(key = lambda x: x[1], reverse=True)
  # writing in the file
  for sim_score in sim_scores:
    rank += 1
    writefile.write(str(i+1)+ ' Q0 '+ str(sim_score[0]) + ' '+str(rank) +' '+ str(sim_score[1])+ ' myRun \n')


writefile.close()


0



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48


Experiment 2

In [ ]:
import gensim
import gensim.downloader as api
model = api.load("glove-twitter-200")
# storing token for each word after exculding the document id
token_sent = [line[1:] for line in preprocessedLines]
# glove_vectors['man']

[==============================================----] 92.5% 701.9/758.5MB downloaded


In [ ]:
# query_pr = preprocess(queries)
from scipy import spatial
import numpy as np
writefile =  open('exp2.txt', 'w')

k = 0
for query in queries:
  k+=1
  # print(k)
  sim_scores = []
# processing a query
  que_tok = process_query(query)
# creating a embedding for each word present in the query, it produces 220-dimensional vector for each word
  que_emd = [model[token] for token in que_tok if token in model]
  # meaning the results over all dimensions
  que_emd =np.mean( np.array(que_emd), axis=0 )
# iterating over all documents
  for i in range(len(token_sent)):
    # if no common words are found between test query and curren document, it will skip
    if len(set( token_sent[i] ) & set( que_tok ) ) / (len(que_tok))<1:
      continue 
    # embeddings for the current document
    line_emd = [model[token] for token in token_sent[i] if token in model]
    # meaning the embedding
    line_emd = np.mean( np.array(line_emd), axis=0 )
    # finding a similarity score and append it to the list
    sim_scores.append([preprocessedLines[i][0], 1- spatial.distance.cosine(line_emd, que_emd) ])
  # sorting the scores
  sim_scores.sort(key = lambda x: x[1], reverse=True)
  rank = 0
  # storing first 1000 document to an output file
  for sim_score in sim_scores[:1000]:
    rank += 1
    writefile.write(str(k)+ ' Q0 '+ str(sim_score[0]) + ' '+str(rank) +' '+ str(sim_score[1])+ ' myRun \n')

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
